# Langchain crash course

In [1]:
import os
from pprint import pprint
import json

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

## LLMs

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Taj Mahal Spice Co."


In [4]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n"Spice and Splendor: Fine Indian Cuisine" '

## Prompt Templates

In [5]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))

{
    "cuisine": "Mexican",
    "text": "\n\n\"El Jard\u00edn de Sabores\" (The Garden of Flavors)"
}


In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.
{
    "cuisine": "Mexican",
    "text": "\n\n\"El Sabor Exquisito\""
}


In [8]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [9]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

results = chain.invoke("Indian")
print(results["input"])
pprint(results["output"])

Indian
('\n'
 '\n'
 '1. Spicy Chicken Tikka Masala\n'
 '2. Vegetable Samosas\n'
 '3. Lamb Vindaloo\n'
 '4. Tandoori Shrimp\n'
 '5. Palak Paneer (spinach and cheese curry)\n'
 '6. Aloo Gobi (potato and cauliflower curry)\n'
 '7. Bhindi Masala (spicy okra)\n'
 '8. Naan bread with garlic or chili\n'
 '9. Mango Lassi (yogurt drink)\n'
 '10. Chai Tea\n'
 '11. Chicken Biryani\n'
 '12. Achari Paneer (pickled cheese curry)\n'
 '13. Malai Kofta (vegetable and cheese dumplings in creamy sauce)\n'
 '14. Tandoori Chicken\n'
 '15. Masala Chai Ice Cream for dessert.')


#### Sequential Chain

In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [12]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [13]:
results = chain.invoke({"cuisine": "Indian"})
pprint(results["cuisine"])
pprint(results["restaurant_name"])
pprint(results["menu_items"])

'Indian'
'\n\n"Royal Masala" '
('\n'
 '1. Chicken Tikka Masala\n'
 '2. Vegetable Korma\n'
 '3. Lamb Vindaloo\n'
 '4. Palak Paneer\n'
 '5. Garlic Naan\n'
 '6. Tandoori Chicken\n'
 '7. Samosas\n'
 '8. Chana Masala\n'
 '9. Butter Chicken\n'
 '10. Aloo Gobi\n'
 '11. Mango Lassi\n'
 '12. Malai Kofta\n'
 '13. Dal Makhani\n'
 '14. Biryani\n'
 '15. Gulab Jamun (dessert)')


## Agents

In [14]:
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')

#### serpapi and llm-math tool

In [15]:
from langchain import hub
from langchain.agents import AgentType, create_react_agent, load_tools, AgentExecutor

In [16]:
prompt = hub.pull("hwchase17/react")

In [17]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "What was the GDP of US in 2022?"

# Let's test it out!
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should search for the answer using a search engine.
Action: Search
Action Input: "GDP of US in 2022"25439.70 USD Billion I should use a calculator to convert the result to a more readable format.
Action: Calculator
Action Input: 25439.70 * 1000000000Answer: 25439700000000.0 I now know the final answer.
Final Answer: The GDP of US in 2022 is 25.44 trillion USD.

> Finished chain.


{'input': 'What was the GDP of US in 2022?',
 'output': 'The GDP of US in 2022 is 25.44 trillion USD.'}

#### Wikipedia and llm-math tool

In [18]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "When was Elon musk born? What is the square root of his age as of 2023?"

# Let's test it out!
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should use Wikipedia to find out when Elon Musk was born and then use a calculator to find the square root of his age.
Action: Wikipedia
Action Input: Elon MuskPage: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acq

{'input': 'When was Elon musk born? What is the square root of his age as of 2023?',
 'output': "The square root of Elon Musk's age as of 2023 is approximately 7.21."}

In [19]:
7.211102550927978 ** 2

51.99999999999999

## Memory

#### ConversationBufferMemory

In [20]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\n"El Sabor Exquisito" (The Exquisite Flavor)'}


In [21]:
name = chain.invoke("Arabic")
print(name)

{'cuisine': 'Arabic', 'history': 'Human: Mexican\nAI: \n\n"El Sabor Exquisito" (The Exquisite Flavor)', 'text': '\n\n"Al-Fawarish" (meaning "The Treasures" in Arabic)'}


In [22]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [23]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [24]:
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': " The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated Australia in the final by 17 runs. The tournament was hosted by England, with a total of eight teams participating. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Gordon Greenidge. The final was held at Lord's Cricket Ground in London, England. It was a historic moment for the sport of cricket and the West Indies team was hailed as champions."}

In [25]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated Australia in the final by 17 runs. The tournament was hosted by England, with a total of eight teams participating. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Gordon Greenidge. The final was held at Lord's Cricket Ground in London, England. It was a historic moment for the sport of cricket and the West Indies team was hailed as champions.",
 'response': ' The answer to 5+5 is 10. This is a basic arithmetic problem and the result is a simple addition of the two numbers. '}

In [26]:
convo.invoke("Who was the captain ofthe winning team?")

{'input': 'Who was the captain ofthe winning team?',
 'history': "Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated Australia in the final by 17 runs. The tournament was hosted by England, with a total of eight teams participating. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Gordon Greenidge. The final was held at Lord's Cricket Ground in London, England. It was a historic moment for the sport of cricket and the West Indies team was hailed as champions.\nHuman: How much is 5+5?\nAI:  The answer to 5+5 is 10. This is a basic arithmetic problem and the result is a simple addition of the two numbers. ",
 'response': ' The captain of the winning team was Clive Lloyd. He was a legendary cricketer for the West Indies team and led them to two consecutive World Cup victories in 1975 and 1979. He was known for his strong leadership skills and his powerful batt

In [27]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated Australia in the final by 17 runs. The tournament was hosted by England, with a total of eight teams participating. The West Indies team was led by Clive Lloyd and had players such as Viv Richards and Gordon Greenidge. The final was held at Lord's Cricket Ground in London, England. It was a historic moment for the sport of cricket and the West Indies team was hailed as champions.
Human: How much is 5+5?
AI:  The answer to 5+5 is 10. This is a basic arithmetic problem and the result is a simple addition of the two numbers. 
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team was Clive Lloyd. He was a legendary cricketer for the West Indies team and led them to two consecutive World Cup victories in 1975 and 1979. He was known for his strong leadership skills and his powerful batting style. He was also known as th

#### ConversationBufferWindowMemory

In [28]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': ' The first cricket world cup was held in 1975 and was won by the West Indies. The event took place in England and was hosted by the International Cricket Council. The West Indies defeated Australia in the final match by 17 runs. It was a historic moment for the West Indies as they became the first ever winners of the cricket world cup.'}

In [29]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': 'Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was held in 1975 and was won by the West Indies. The event took place in England and was hosted by the International Cricket Council. The West Indies defeated Australia in the final match by 17 runs. It was a historic moment for the West Indies as they became the first ever winners of the cricket world cup.',
 'response': '  5+5 is equal to 10.'}

In [30]:
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: How much is 5+5?\nAI:   5+5 is equal to 10.',
 'response': ' I am not sure which specific team you are referring to. Can you provide more context?'}

## Streaming

In [39]:
for chunk in convo.stream("Write me a song about goldfish on the moon"):
    print(chunk["history"])
    # print(chunk.content, end="", flush=True)

Human: What is your name?
AI:  My specific name is determined by my creators and programmers. They have given me the name AI, but I am constantly learning and evolving, so my abilities and knowledge are always expanding. Is there something else you would like to know?


In [41]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import GPT4All

# Set up the prompt template
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# Set up the local instace of the LLM
local_path = "/Users/brianroepke/Library/Application Support/nomic.ai/GPT4All/mistral-7b-openorca.Q4_0.gguf"  # noqa: E501
callbacks = [StreamingStdOutCallbackHandler()]
memory = ConversationBufferMemory()
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True, streaming=True)  # noqa: E501
# llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

llm_chain = ConversationChain(llm=llm)

In [42]:
for chunk in llm_chain.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)

 Sure! Here's a little ditty I came up with:

Sparkling Water Lullaby
(To the tune of Twinkle, Twinkle Little Star)

Sparkling water so clear and bright,
Bubbles dance in the moonlight,
Quenching thirst on a summer night.

Chorus:
Drink it down, drink it up,
Feel your body tingle with delight!
Sparkling water is so right!

In the meadow where daisies play,
A cool stream flows all day,
Filling glasses for passersby.

Chorus: Drink it down...

On a mountain peak so high,
Icicles glisten in the sky,
Melting into sparkling water.

Chorus: Drink it down...

In the depths of the ocean blue,
A school of fish come swimming through,
Drinking from fountains made for two.

Chorus: Drink it down...

So next time you're parched and dry,
Remember sparkling water's why,
It quenches thirst with every sigh!
{'input': 'Write me a song about sparkling water.', 'history': '', 'response': " Sure! Here's a little ditty I came up with:\n\nSparkling Water Lullaby\n(To the tune of Twinkle, Twinkle Little Star)

In [44]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain


chat_model = ChatOpenAI()
system_message = SystemMessage(content="Act as a Recruiter for a Big Tech Startup")

prompt = ChatPromptTemplate(messages=[system_message])

conversation = ConversationChain(llm=chat_model, prompt=prompt, verbose=True)

def input(human_response):
    return conversation.run(human_response)

ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects [], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)